In [26]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()
# get a handle to the workspace



Workspace name: quick-starts-ws-259362
Azure region: eastus2
Subscription id: 9b72f9e6-56c5-4c16-991b-19c652994860
Resource group: aml-quickstarts-259362


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute



# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.



from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "aml-first-compute"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)
### YOUR CODE HERE ###

Found existing cluster, use it.

Running


In [8]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({'--C':uniform(0.1,1),'--max_iter':choice(50,100,200)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2,slack_factor=1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                            script='train.py',
                            #arguments=['--C', arg1_val, '--arg2', arg2_val],
                            compute_target=cpu_cluster,
                            environment=sklearn_env)
# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,hyperparameter_sampling=ps,
primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,primary_metric_name='Accuracy',max_concurrent_runs=4,max_total_runs=30,policy=policy)



In [9]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
experiment = Experiment(workspace=ws, name='hyperdrive_example')
hyperdrive_run = experiment.submit(hyperdrive_config)
hyperdrive_run.wait_for_completion(show_output=True)


RunId: HD_eae21f0a-8674-4c06-b2d0-49d96a2cc59d
Web View: https://ml.azure.com/runs/HD_eae21f0a-8674-4c06-b2d0-49d96a2cc59d?wsid=/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourcegroups/aml-quickstarts-259362/workspaces/quick-starts-ws-259362&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-05-19T04:41:28.803985][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2024-05-19T04:41:29.3745189Z][SCHEDULER][INFO]Scheduling job, id='HD_eae21f0a-8674-4c06-b2d0-49d96a2cc59d_1' 
[2024-05-19T04:41:29.3009719Z][SCHEDULER][INFO]Scheduling job, id='HD_eae21f0a-8674-4c06-b2d0-49d96a2cc59d_0' 
[2024-05-19T04:41:29.536774][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2024-05-19T04:41:29.5920209Z][SCHEDULER][INFO]Scheduling job, id='HD_eae21f0a-8674-4c06-b2d0-49d96a2cc59d_2' 
[2024-05-19T04:41:29.7090890Z][SCHEDULER][INFO]Scheduling job, id='HD_eae21f0a-8674-4c06-b2d0-49d96a2cc5

ExperimentExecutionException: ExperimentExecutionException:
	Message: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "The output streaming for the run interrupted.\nBut the run is still executing on the compute target. \nDetails for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run"
    }
}

In [10]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(f"Best run arguments: {best_run.get_details()['runDefinition']['arguments']}")
print(f"Best run metrics: {best_run.get_metrics()}")
print(f"Best run file names: {best_run.get_file_names()}")
joblib.dump(value=best_run.id, filename="./outputs/hyperdrive_model.joblib")

Best run arguments: ['--C', '0.7209608007003498', '--max_iter', '200']


KeyboardInterrupt: 

In [11]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
paths="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds=TabularDatasetFactory.from_delimited_files(paths)
### YOUR CODE HERE ###

In [20]:
from train import clean_data
import pandas as pd
# Use the clean_data function to clean your data.
x, y = clean_data(ds)

train_df = pd.concat([x, y], axis=1)
train_df_path = os.path.join(".", 'train.csv')
train_df.to_csv(train_df_path)
data_store = ws.get_default_datastore()
data_store.upload(src_dir=".", target_path=".")
train_ds = TabularDatasetFactory.from_delimited_files(data_store.path("azureml://subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourcegroups/aml-quickstarts-259362/workspaces/quick-starts-ws-259362/datastores/workspaceblobstore/paths/UI/2024-05-19_050201_UTC/bankmarketing_train.csv"))


Uploading an estimated of 18 files


ClientAuthenticationError: Operation returned an invalid status 'Server failed to authenticate the request. Make sure the value of Authorization header is formed correctly including the signature.'
ErrorCode:AuthenticationFailed

In [27]:
datasets = Dataset.get_all(ws)
for name, dataset in datasets.items():
    print(f"Name: {name}, Dataset: {dataset}")


In [22]:
from azureml.core import Workspace, Dataset, Experiment
from azureml.train.automl import AutoMLConfig

dataset = Dataset.get_by_name(ws, name="train_df")

UserErrorException: UserErrorException:
	Message: Cannot find dataset registered with name "train_df" (version: None) in the workspace.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Cannot find dataset registered with name \"train_df\" (version: None) in the workspace."
    }
}

In [30]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=2)


In [ ]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(config=automl_config, show_output=True)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)
automl_run

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_331e1695-c571-4402-9e99-e9d57b26d42c,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

2024/05/19 05:10:07 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2024-05-19:05:38:03,525 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2024-05-19:05:38:05,250 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2024-05-19:05:38:05,251 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2024-05-19:05:38:06,853 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2024-05-19:

In [ ]:
# Retrieve and save your best automl model.

best_run, best_model = automl_run.get_output()
print(best_run)
print(best_model)
print(f"Best run arguments: {best_run.get_details()['runDefinition']['arguments']}")
print(f"Best run metrics: {best_run.get_metrics()}")
print(f"Best run file names: {best_run.get_file_names()}")
joblib.dump(best_model, 'outputs/automl_model.joblib')